In [ ]:
!pip3 install -q tensorflow
!pip3 install -q numpy
!pip3 install -q scipy
!pip3 install -q matplotlib
!pip3 install -q pandas
!pip3 install -q scikit-learn
!pip3 install -q xlrd xlwt
!pip3 install -q openpyxl



In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

data = np.load("dataset2.npy", allow_pickle=True)

dataImages = data[:,0]
dataLabels = data[:,1].astype('uint8')


In [ ]:
dataShape = (len(dataImages),)+dataImages[0].shape
print(dataShape)


In [ ]:
# expand the dim of dataset
a = np.empty(dataShape, dtype=np.uint8)

for i in range(len(dataImages)):
    a[i] = dataImages[i].astype('uint8')

dataImages = a

In [ ]:
# Split the data up in train and test sets
trainImages, testImages, trainLabels, testLabels = train_test_split(dataImages, dataLabels, test_size=0.33, random_state=42)

classNames = ['Clean', 'Contaminated']

In [ ]:
trainImages.shape

In [ ]:
trainImages.dtype

In [ ]:
trainImages[0]

In [ ]:
len(trainLabels)

In [ ]:
trainLabels

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(trainImages[0])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
trainImages = trainImages / 255.0

testImages = testImages / 255.0

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(trainImages[i], cmap=plt.cm.binary)
    plt.xlabel(classNames[trainLabels[i]])
plt.show()

In [ ]:
size = trainImages[0].shape
size

In [ ]:
import tensorflow as tf


model = tf.keras.Sequential([
    # tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Flatten(input_shape=size),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(trainImages, trainLabels, epochs=10)

In [ ]:
testLoss, testAcc = model.evaluate(testImages,  testLabels, verbose=2)

print('\nTest accuracy:', testAcc)

In [ ]:
probabilityModel = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
predictions = probabilityModel.predict(testImages)

In [ ]:
predictions[0]

In [ ]:
np.argmax(predictions[0])

In [ ]:
testLabels[0]

In [ ]:
def plot_image(i, predictionsArray, trueLabels, img):
  trueLabel, img = trueLabels[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predictedLabel = np.argmax(predictionsArray)
  if predictedLabel == trueLabel:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(classNames[predictedLabel],
                                100*np.max(predictionsArray),
                                classNames[trueLabel]),
                                color=color)

def plot_value_array(i, predictionsArray, trueLabels):
  trueLabel = trueLabels[i]
  plt.grid(False)
  plt.xticks(range(2))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictionsArray, color="#777777")
  plt.ylim([0, 1])
  predictedLabel = np.argmax(predictionsArray)

  thisplot[predictedLabel].set_color('red')
  thisplot[trueLabel].set_color('blue')

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], testLabels, testImages)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  testLabels)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], testLabels, testImages)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  testLabels)
plt.show()

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions[i], testLabels, testImages)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions[i], testLabels)
plt.tight_layout()
plt.show()

In [ ]:
# Grab an image from the test dataset.
img = testImages[1]

print(img.shape)

In [ ]:
# Add the image to a batch where it's the only member.
img = (np.expand_dims(img,0))

print(img.shape)
print(img)

In [ ]:
predictionsSingle = probabilityModel.predict(img)

print(predictionsSingle)

In [ ]:
plot_value_array(1, predictionsSingle[0], testLabels)
_ = plt.xticks(range(10), classNames, rotation=45)
plt.show()

In [ ]:
np.argmax(predictionsSingle[0])

In [ ]:
import os

modelDir = 'tf_model'
version = "1"

exportPath = os.path.join(modelDir, version)

# save the model
model.save(exportPath, save_format="tf")

print('\nexport path = {}'.format(exportPath))

!dir {exportPath}

In [ ]:
import json
# create json object

jsonData = json.dumps({"signature_name":"serving_default", "instances":testImages[:2].tolist()})

In [ ]:
import requests
headers = {"content-type":"application/json"}
json_response = requests.post('http://localhost:8501/v1/models/giwa:predict', data=jsonData, headers=headers)

predictions = json.loads(json_response.text)['predictions']

In [ ]:
pred = [np.argmax(predictions[p]) for p in range(len(predictions))]
print("Predictions: ", pred)
print("Actual: ", testLabels[:2].tolist())